In [1]:
import os
import json
import pandas as pd
import traceback

In [11]:
from langchain_openai import ChatOpenAI


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
KEY=os.getenv("OPENAI_API_KEY")

In [16]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [17]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [21]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [22]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [23]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [24]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True) 

C:\Users\HP\AppData\Local\Temp\ipykernel_2800\2575176920.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)


In [25]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [26]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [28]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [29]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"],verbose=True)

In [34]:
file_path = r"C:\Users\HP\mcqgenerator\data.txt"

In [35]:
with open(file_path, "r") as file:
    TEXT= file.read()

In [36]:
print(TEXT)

Machine learning (ML) is a subfield of artificial intelligence (AI) that focuses on developing algorithms and models that allow computers to learn from and make predictions or decisions based on data, without being explicitly programmed for every task. In essence, it empowers machines to learn from experience and improve their performance over time by analyzing data and identifying patterns. 
Here's a more detailed explanation:
Core Concept:
Machine learning algorithms are designed to find patterns and relationships within data, and then use those patterns to make predictions or decisions about new, unseen data. 
Instead of being explicitly programmed with specific instructions, machines learn from data through a process of training. 
This training process involves feeding the algorithm a large dataset and allowing it to adjust its internal parameters to minimize errors and improve accuracy. 
Key Aspects:
Data-driven:
Machine learning relies heavily on data. The quality and quantity of

In [37]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [38]:
NUMBER = 3
SUBJECT = "Machine Learning"
TONE = "simple"


In [39]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json": json.dumps(RESPONSE_JSON)

        }
    )

C:\Users\HP\AppData\Local\Temp\ipykernel_2800\815328654.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a subfield of artificial intelligence (AI) that focuses on developing algorithms and models that allow computers to learn from and make predictions or decisions based on data, without being explicitly programmed for every task. In essence, it empowers machines to learn from experience and improve their performance over time by analyzing data and identifying patterns. 
Here's a more detailed explanation:
Core Concept:
Machine learning algorithms are designed to find patterns and relationships within data, and then use those patterns to make predictions or decisions about new, unseen data. 
Instead of being explicitly programmed with specific instructions, machines learn from data through a process of training. 
This training process involves feeding the algorithm a large dataset and allowing it to adjust its internal parameters to minimize errors and impro

In [40]:
response

{'text': "Machine learning (ML) is a subfield of artificial intelligence (AI) that focuses on developing algorithms and models that allow computers to learn from and make predictions or decisions based on data, without being explicitly programmed for every task. In essence, it empowers machines to learn from experience and improve their performance over time by analyzing data and identifying patterns. \nHere's a more detailed explanation:\nCore Concept:\nMachine learning algorithms are designed to find patterns and relationships within data, and then use those patterns to make predictions or decisions about new, unseen data. \nInstead of being explicitly programmed with specific instructions, machines learn from data through a process of training. \nThis training process involves feeding the algorithm a large dataset and allowing it to adjust its internal parameters to minimize errors and improve accuracy. \nKey Aspects:\nData-driven:\nMachine learning relies heavily on data. The quali

In [41]:
quiz=response.get("quiz")

In [ ]:
quiz=json.loads(quiz)

{'1': {'mcq': 'What is the core concept of machine learning?',
  'options': {'a': 'Explicit programming with specific instructions',
   'b': 'Learning from data and finding patterns',
   'c': 'Analyzing data without using algorithms',
   'd': 'Ignoring data and making random decisions'},
  'correct': 'b'},
 '2': {'mcq': 'Which industry uses machine learning for diagnosing diseases and predicting patient outcomes?',
  'options': {'a': 'Finance',
   'b': 'E-commerce',
   'c': 'Healthcare',
   'd': 'Transportation'},
  'correct': 'c'},
 '3': {'mcq': 'How do machine learning algorithms learn from data?',
  'options': {'a': 'Through trial and error',
   'b': 'Using labeled data',
   'c': 'Ignoring data completely',
   'd': 'By not adjusting internal parameters'},
  'correct': 'b'}}

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S').log

'06_29_2025_17_39_41'